In [152]:
import pandas
import nltk
import textblob

nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('brown')

[nltk_data] Downloading package words to
[nltk_data]     /Users/savannadenega/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/savannadenega/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/savannadenega/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     /Users/savannadenega/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [153]:
# Reading the dataset
data = pandas.read_csv('dataset/review_hotel.csv')
data_analyze = pandas.read_csv('dataset/review_hotel_analyze.csv')

In [154]:
# Exploring the top 5 observations
data.head()

,Review
0,I am so angry that i made this post available...
1,Rooms are nice but for elderly a bit difficul...
2,My room was dirty and I was afraid to walk ba...
3,You When I booked with your company on line y...
4,Backyard of the hotel is total mess shouldn t...


In [155]:
# Exploring the last 5 observations.
data.tail()

,Review
338,Bed was really comfortable
339,Beautiful hotel with views of the park Transp...
340,No Positive
341,Large clean rooms with a nice size comfy bed ...
342,Well equipped nice room Traditinal but modern...


In [156]:
# Number of dataset consumed itens
number_dataset_consumed_itens = 2

sentences_tokens = []

# Set language stopwords to english
stop_words = set(nltk.corpus.stopwords.words('english')) 

# Do tokenization over each Review from dataset
for index, row in data.head(number_dataset_consumed_itens).iterrows():
    sentences_tokens.append(nltk.word_tokenize(row['Review']))

# Put all words on singular and after find infinitive verbs of words

word_capital_letter = ''
lemmatized = ''
index = -1

for row_tokenizing in sentences_tokens:
    for token in row_tokenizing:
        index += 1
        word_capital_letter = token
        lemmatized = nltk.WordNetLemmatizer().lemmatize(word_capital_letter.lower(), 'v')
        if word_capital_letter.istitle():
            word_capital_letter = lemmatized.capitalize()
        else:
            word_capital_letter = lemmatized
        token = word_capital_letter
        row_tokenizing[index] = token
    
    index = -1           

# Remove stop words

index = -1

for words_sentence in sentences_tokens:
    for word in words_sentence:
        index += 1
        if word in stop_words:
            del words_sentence[index]
    
    index = -1
    
print(sentences_tokens)

[['I', 'so', 'angry', 'i', 'make', 'post', 'available', 'via', 'possible', 'sit', 'use', 'plan', 'trip', 'no', 'one', 'make', 'mistake', 'book', 'place', 'I', 'make', 'book', 'via', 'book', 'com', 'We', 'stay', '6', 'nights', 'this', 'hotel', '11', '17', 'July', 'Upon', 'arrival', 'be', 'place', 'a', 'small', 'room', 'the', '2nd', 'floor', 'the', 'hotel', 'It', 'turn', 'that', 'be', 'the', 'room', 'book', 'I', 'specially', 'reserve', '2', 'level', 'duplex', 'room', 'that', 'would', 'a', 'big', 'windows', 'high', 'ceilings', 'The', 'room', 'be', 'ok', 'you', 't', 'mind', 'break', 'window', 'can', 'be', 'close', 'hello', 'rain', 'a', 'mini', 'fridge', 'contain', 'sort', 'a', 'bio', 'weapon', 'least', 'guess', 'by', 'smell', 'it', 'I', 'intimately', 'ask', 'change', 'room', 'after', 'explain', '2', 'time', 'i', 'book', 'duplex', 'btw', 'cost', 'same', 'a', 'simple', 'double', 'get', 'way', 'volume', 'due', 'the', 'high', 'ceiling', 'offer', 'room', 'only', 'next', 'day', 'i', 'to', 'check

<span style="color:orange">d) Definir as atividades de aprendizagem de máquina a utilizar.<span/>

<span style="color:orange">- Análise pos/neg com biblioteca textblob<span/>

In [157]:

polarity_sentences = []

for sentence_token in sentences_tokens:
    
    text = nltk.tokenize.treebank.TreebankWordDetokenizer().detokenize(sentence_token)
    
    blob = textblob.TextBlob(text)

    # [('The', 'DT'), ('titular', 'JJ'),
    #  ('threat', 'NN'), ('of', 'IN'), ...]
    blob.tags

    # WordList(['titular threat', 'blob',
    #            'ultimate movie monster',
    #            'amoeba-like mass', ...])
    blob.noun_phrases  

    if index_polarity_sentences < len(sentences_tokens):
        for sentence in blob.sentences:
            polarity_sentences.append(sentence.sentiment.polarity)
    
    
print(polarity_sentences)


[0.05578940667175961, 0.032653061224489785]


<span style="color:orange">e) Definir as métricas utilizadas para avaliação.<span/>

<span style="color:orange">- Realizar classificação manual (positiva e negativa) sobre cada frase e adicionar a um arquivo para os experimentos<span/>

In [158]:
analyze_sentences_tokens = []

for index, row in data_analyze.head(number_dataset_consumed_itens).iterrows():
    analyze_sentences_tokens.append(nltk.word_tokenize(row['Analyze']))
    

analyze_sentences = []

for analyze_sentence_tokens in analyze_sentences_tokens:
    
    text = nltk.tokenize.treebank.TreebankWordDetokenizer().detokenize(analyze_sentence_tokens)
    analyze_sentences.append(text)
    
print(analyze_sentences)

['Negative', 'Positive']


<span style="color:orange">f) Realizar experimentos. <span/>

<span style="color:orange">- Realizar análise entre a resposta do nosso código e a classificação manual (positiva e negativa) realizada em cada frase<span/>

In [160]:
index = 0

for polarity_sentence in polarity_sentences:  
    if polarity_sentence > 0 and analyze_sentences[index] == "Positive":
        print("Sentence Number: " + str(index) + " - Comparison Code X Manual Analysis: SUCCESS")
    elif polarity_sentence < 0 and analyze_sentences[index] == "Negative":
        print("Sentence Number: " + str(index) + " - Comparison Code X Manual Analysis: SUCCESS")
    else:
        print("Sentence Number: " + str(index) + " - Comparison Code X Manual Analysis: FAILURE")
    index = index + 1
        

Sentence Number: 0 - Comparison Code X Manual Analysis: FAILURE
Sentence Number: 1 - Comparison Code X Manual Analysis: SUCCESS


<span style="color:orange">g) Relatar os resultados segundo estas etapas previstas e escrever uma conclusão sobre o experimento, comentando sobre:<span/>


<span style="color:orange">a. as ferramentas e recursos utilizados;<span/>


<span style="color:orange">b. as referências (sites, livros, artigos) utilizados;<span/>


<span style="color:orange">c. os resultados obtidos com o experimento;<span/>


<span style="color:orange">d. possibilidades de trabalhos futuros com este experimento;<span/>


<span style="color:orange">e. aspectos do aprendizado pessoal durante o experimento.<span/>

#  # TODO